<a href="https://colab.research.google.com/github/soueuwilliam/Estudos-R/blob/main/IBS/Aulas/Aula07_STRINGdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Coloque o caminho do arquivo R para lê
caminho_arquivo_r <- '/content/string.R'
arquivo <- readLines(caminho_arquivo_r)
writeLines(arquivo)

In [ ]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

In [26]:
# Instalar pacotes requeridos, se não estiverem instalados:
# RCurl
if(!require(httr)) install.packages("httr")
#igraph
if(!require(igraph)) install.packages("igraph")
# RedeR
if(!require(RedeR)) BiocManager::install("RedeR")

In [29]:
# Carregar pacotes
library(httr)
library(igraph)
library(RedeR)


In [ ]:
# STRING db ---------------------------------------------------------------
# PASSO 1 - OBTER A REDE DE INTERAÇÃO DO STRING ----

# Importar tabela com a lista de genes
aponer <- read.table(file = "apo_dnadamage.txt", header = TRUE, stringsAsFactors = FALSE)
head(aponer)

In [34]:
# Para obtermos as informações do STRING, iremos usar sua API. Esta interface possui métodos
# para a obtenção dos dados disponíveis no banco (Para saber mais sobre cada método,
# visite https://string-db.org/help/api/).

# O primeiro método a ser usado é 'get_string_ids', que irá mapear uma lista de
# genes para os identificadores próprios do STRING (Para saber quais os parâmetros do método,
# visite https://string-db.org/cgi/help.pl?subpage=api%23mapping-identifiers).

# Para fazer este mapeamento, quando submetemos uma requisição à API do string programaticamente,
# temos que concatenar os identificadores e separá-los com o símbolo "%0d".
identifiers_gene_name <- paste0(unique(aponer$Approved_Symbol), collapse = "%0d")

# Fazer a solicitação a API do STRING
req <- httr::POST(
  "https://string-db.org/api/tsv/get_string_ids",
  body = list(
    identifiers = identifiers_gene_name,
    echo_query = "1",
    species = "9606"
  )
)
map_ids <- read.table(text = content(req, "text"), sep = "\t", header = T, quote = "")

In [62]:

# Agora, de posse dos identificadores do STRING, vamos usá-los para obtermos
# a rede de interação entre estes genes. Usaremos o método
# 'network' (https://string-db.org/cgi/help.pl?subpage=api%23getting-the-string-network-interactions),
identifiers_string_id <- paste0(unique(map_ids$stringId), collapse = "%0d")
req <- httr::POST(
  "https://string-db.org/api/tsv/network",
  body = list(
    identifiers = identifiers_string_id,
    # identificadores do stringID, obtidos na etapa anterior
    required_core = "0",
    # score mínimo para cada interação
    species     = "9606" # espécie (H. sapiens)
  )
)
int_network <- read.table(text = content(req, "text"), sep = "\t", header = T)
int_network <- unique(int_network)


In [63]:
# A tabela conterá as seguintes colunas:
# stringId_A -	STRING identifier (protein A)
# stringId_B - STRING identifier (protein B)
# preferredName_A	- common protein name (protein A)
# preferredName_B	- common protein name (protein B)
# ncbiTaxonId	- NCBI taxon identifier
# score	- combined score
# nscore - gene neighborhood score
# fscore - gene fusion score
# pscore - phylogenetic profile score
# ascore - coexpression score
# escore - experimental score
# dscore - database score
# tscore - textmining score

# Função para combinar os scores de acordo com o algoritmo usado pelo STRING
combinescores <- function(dat, evidences = "all", confLevel = 0.4){
  if(evidences[1] == "all"){
    edat<-dat[,-c(1,2,ncol(dat))]
  } else {
    if(!all(evidences%in%colnames(dat))){
      stop("NOTE: one or more 'evidences' not listed in 'dat' colnames!")
    }
    edat<-dat[,evidences]
  }
  if (any(edat > 1)) {
    edat <- edat/1000
  }
  edat<-1-edat
  sc<- apply(X = edat, MARGIN = 1, FUN = function(x) 1-prod(x))
  dat <- cbind(dat[,c(1,2)],combined_score = sc)
  idx <- dat$combined_score >= confLevel
  dat <-dat[idx,]
  return(dat)
}



In [64]:
# Selecionar os scores relativos apenas aos canais de coexpressão, experimentos e database e
# combinar seus scores. Selecionar as interações cujos scores são maiores que 0.9.
int_network <- combinescores(int_network, evidences = c("ascore", "escore", "dscore"), confLevel = 0.9)



In [65]:
# PASSO 2 - PREPROCESSAR A REDE ----

# Remover o identificador de espécie em cada ENSP
int_network$stringId_A <- substring(int_network$stringId_A, 6, 1000)
int_network$stringId_B <- substring(int_network$stringId_B, 6, 1000)

In [66]:

# Filtrar a rede, mantendo apenas os ENSP que estão presentes no nosso dataframe inicial
idx1 <- int_network$stringId_A %in% aponer$ENSP
idx2 <- int_network$stringId_B %in% aponer$ENSP
int_network <- int_network[idx1 & idx2,]
rownames(aponer) <- aponer$ENSP


In [67]:
# Manter somente os nomes dos genes na tabela de interação
int_network$gene_name_A <- aponer[int_network$stringId_A, "Approved_Symbol"]
int_network$gene_name_B <- aponer[int_network$stringId_B, "Approved_Symbol"]
int_network <- int_network[, c("gene_name_A", "gene_name_B", "combined_score")]

In [72]:
# PASSO 3 - PLOTAR A REDE DE INTERAÇÃO ----

# Combinar as vias em um vetor
aponer$vias <- NULL
aponer$vias[aponer$Apoptosis == 1] <- 1
aponer$vias[aponer$DNAdamage == 1] <- 2
idx <- rowSums(aponer[, c("Apoptosis","DNAdamage")]) > 1
aponer$vias[idx] <- 3

In [ ]:
aponer

In [79]:
# Criar o grafo g com base na informacao em aponer.inter
g <- graph_from_data_frame(int_network, directed=FALSE)

In [80]:
# Transferir os atributos dos nós de 'aponer' para o grafo criado (referência na coluna 1)
g <- att.mapv(g, dat = aponer, refcol = 1)

In [81]:

# Mapear atributos de nós em escala de cores
cols <- c("green","red","grey")
g <- att.setv(g, from = "vias", to = "nodeColor", cols=cols)

In [82]:
# Definir outros atributos do gráfico
# att.setv() # imprime atributos validos do RedeR
V(g)$nodeLineColor <- "black"
V(g)$nodeLineWidth <- 2
V(g)$nodeFontSize <- 16

In [83]:
# att.sete() #imprime atributos validos do RedeR
E(g)$edgeColor <- "grey50"
E(g)$edgeWidth <- 2

In [84]:
# Abrir porta para o RedeR e adicionar o grafo
rdp <- RedPort()
calld(rdp)
addGraph(rdp, g)


................................................................................



The Java interface is not responding to initialization!

Please, check whether Java is installed in your machine.

RedeR will need Java Runtime Environment (Java >=11)


For a general diagnosis, re-run the 'calld' function with 'checkcalls=TRUE',
for example: > calld(rdp, checkcalls=TRUE)



In [ ]:
# Adicionar legenda
addLegend.color(rdp,
                colvec = cols,
                labvec = c("Apoptosis", "DNAdamage", "Intersection"),
                title = "Pathway")

In [ ]:

# Plotar a rede com o ggraph ----------------------------------------------

if(!require(ggraph)) install.packages("ggraph")
if(!require(tidygraph)) install.packages("tidygraph")

library(ggraph)
library(tidygraph)

In [ ]:
# Obter um dataframe de referência, somente com os genes que participam da rede
ref <- aponer[aponer$Approved_Symbol %in% unique(c(int_network$gene_name_A,int_network$gene_name_B)),]


In [ ]:
# Colocar nome para cada uma das vias
ref$vias <- factor(ref$vias, levels = 1:3, labels = c("Apoptosis", "DNAdamage", "Both"))

In [ ]:
# Criar rede
gr <- tbl_graph(nodes = ref, edges = int_network, directed = F, node_key = "Approved_Symbol")

In [ ]:
# Plotar rede
ggraph(gr, layout = "nicely" ) +
  geom_edge_link(alpha = 0.5) +
  geom_node_point(aes(col = vias)) +
  geom_node_text(aes(label = Approved_Symbol), cex = 1.3)